In [28]:
import pandas as pd

# https://ca.investing.com/indices/us-spx-500-historical-data 
df = pd.read_csv('./Datasets/S&P500-USA.csv', thousands=",")

pattern = r'\d\d\d\d-01-01'
df = df[df['Date'].str.contains(pattern)]
df = df.reset_index(drop=True).iloc[::-1]

df['YoY Change'] = df['Price'].pct_change()*100
df.drop(columns=['Vol.'], inplace=True)
df.dropna(inplace=True)

df.to_csv('./Datasets/S&P500-USA-Clean.csv')